# prep

In [1]:
from deap import base
from deap import creator
from deap import tools

import pandas as pd
import random
import numpy as np
from sympy import *
from graphviz import *

import matplotlib.pyplot as plt
import seaborn as sns
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
from sklearn.model_selection import train_test_split
import statsmodels.api as sm 

def pow_3(x1):
    f = x1**3
    return f

def protected_division(x,y):
    try:
        return x/y
    except ZeroDivisionError:
        return 0

pow_3 = make_function(function=pow_3, name='pow3', arity=1)
protected_division = make_function(function=protected_division, name='protected_division', arity=2)

converter = {
    'sub': lambda x, y : x - y,
    #'div': lambda x, y : x/y,
    'mul': lambda x, y : x*y,
    'add': lambda x, y : x + y,
    'neg': lambda x    : -x,
    'pow': lambda x, y : x**y,
    'sin': lambda x    : sin(x),
    'cos': lambda x    : cos(x),
    'log': lambda x    : ln(x),
    'inv': lambda x: 1/x,
    'sqrt': lambda x: x**0.5,
    'pow3': lambda x: x**3,
    'protected_division': lambda x: x/y,
}

<ipython-input-1-320612cb3c73>:24: RuntimeWarning: invalid value encountered in true_divide
  return x/y


ValueError: supplied function protected_division does not have closure against zeros in argument vectors.

In [ ]:
nsample = 400
sig = 0.2
x = np.linspace(-50, 50, nsample)
X = np.column_stack((x/5, 10*np.sin(x), (x-5)**3, np.ones(nsample)))
beta = [0.01, 1, 0.001, 5.]
y_true = np.dot(X, beta)
y = y_true + sig * np.random.normal(size=nsample)
df = pd.DataFrame()
df['x']=x
df['y']=y

y = df['y']
X = df[['x']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# , pow_3
# First Test
function_set = ['add', 'sub', 'mul', protected_division, 'cos', 'sin', 'neg', 'inv', 'log', pow_3]

est_gp = SymbolicRegressor(
    population_size=5000
    ,metric="mean absolute error" # rmse
    ,function_set=function_set
    ,generations=20
    ,stopping_criteria=0.01
    ,p_crossover=0.7
    ,p_subtree_mutation=0.1
    ,p_hoist_mutation=0.05
    ,p_point_mutation=0.1
    ,max_samples=0.9
    ,verbose=1
    ,parsimony_coefficient=0.001
    ,random_state=0
    ,feature_names=X_train.columns
)
                          
est_gp.fit(X_train, y_train)
print('R2:',est_gp.score(X_test,y_test))
next_e = sympify((est_gp._program), locals=converter)
next_e   

print(est_gp._program) 
df['prediction'] = est_gp.predict(X)
df.to_csv('c:/data/test.csv', index=False)

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    12.62      3.4096e+134        4          16.1472          10.9212      4.41m
   1    10.06      5.75637e+40       12          14.7471          19.1889      4.39m
   2     8.78      7.11374e+13        4          14.8138          22.9223      3.51m
   3     8.88      2.94318e+23       12          10.8084          11.3534     14.58m
   4     8.06      4.92231e+40       15          7.20635          7.25754     17.75m
   5     9.50      2.71409e+37       15          7.11218          8.10509     16.72m
   6    12.41      3.36796e+37       23          6.89953          7.04161     14.87m
   7    14.49      5.50037e+45       13          6.89445          8.65066     14.81m
   8    17.30      2.92384e+11       34          6.54792          8.34345  

<ipython-input-91-123b1e43a488>:19: RuntimeWarning: overflow encountered in power
  f = x1**3


  22   101.21      8.96104e+34       92          3.38467          4.24152    408.94m


c:\python\python39\lib\site-packages\gplearn\functions.py:46: RuntimeWarning: invalid value encountered in cos
  return self.function(*args)
c:\python\python39\lib\site-packages\numpy\lib\function_base.py:412: RuntimeWarning: invalid value encountered in multiply
  avg = np.multiply(a, wgt, dtype=result_dtype).sum(axis)/scl


  23   111.76              inf      101          3.32124          4.77872     38.96m
  24   111.23      6.97595e+45      108          3.28407          5.53462     34.90m
  25   104.94      3.41498e+47      187          3.19965          5.61622     33.12m
  26   102.78      4.11879e+51      154          3.20383          5.43902     32.78m
  27   100.92      1.26355e+40      115          3.08999          5.79429     31.40m
  28   102.99     7.43854e+128      145          3.13566          5.43759     30.98m
  29   104.82      9.22718e+70      246          3.10647          5.54806     31.30m
  30   105.75      2.62618e+65      146          3.15776          4.49127     31.10m
  31   104.16      6.39981e+40      127          3.06107          5.10754     31.14m
  32   106.29              nan       98              nan              nan     30.05m
  33   108.46              nan       86              nan              nan     29.84m


...

  35   118.54              nan      197              nan              nan     30.97m


c:\python\python39\lib\site-packages\gplearn\functions.py:127: RuntimeWarning: overflow encountered in true_divide
  return np.where(np.abs(x2) > 0.001, np.divide(x1, x2), 1.)


  36   134.04              nan      161              nan              nan     34.66m
  37   146.00              nan       88              nan              nan     36.10m
  38   157.26              nan      107              nan              nan     37.37m


c:\python\python39\lib\site-packages\gplearn\functions.py:144: RuntimeWarning: overflow encountered in true_divide
  return np.where(np.abs(x1) > 0.001, 1. / x1, 0.)


  39   168.96              nan      204              nan              nan     39.47m
  40   183.17              nan      250              nan              nan     41.19m
  41   194.67              nan      141              nan              nan     42.66m
  42   205.59              nan      232              nan              nan     43.58m
  43   219.79              nan      369              nan              nan     45.04m
  44   232.42              nan       93              nan              nan     46.97m
  45   243.52              nan      229              nan              nan     48.54m
  46   257.26              nan      167              nan              nan     50.76m
